In [1]:
# df_players
# name, elo, state (H/m), move time avg (not possible yet), acpl game avg
# df_games
# p1_ID, p2_ID, winner_ID, date, opening, p1_ELO, p2_ELO
# df_moves
# player_ID, game_ID, time, eval, fen/move, comment

In [2]:
import pandas as pd
import numpy as np
import uuid        # for id generation
import shortuuid

import chess.pgn
from stockfish import Stockfish

In [3]:
# stockfish = Stockfish(
#     'stockfish_14_linux_x64/stockfish_14_linux_x64/stockfish_14_x64', 
#     parameters={"Threads": 2, 'Min Split Depth': 26, 'Ponder':True}
# )
# stockfish.set_elo_rating(2600)
# stockfish.set_skill_level(30)

In [4]:
%%time

players = {
    'White':[],
    'White_Elo': [],
    'Black': [],
    'Black_Elo': [],
    'WhiteIsComp':[],
}

games = {
    'Game_ID': [],
    'Date' : [],
    'White':[],  # Dummy ID
    'White_Elo': [],
    'Black': [],  # Dummy ID
    'Black_Elo': [],
    'ECO': [],
    'Result': [],
}

moves_log_dict = {
    'Game_ID': [],
    'White': [],  # Dummy ID
    'Black': [],  # Dummy ID
    'ECO': [],
    'FEN_moves': [],
    #'cpl': [],
    'WhiteIsComp': [],
    'Result': [],
}

# read file
pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8')  # always a Comp vs Player
game_counter = 0

while True:  # keep reading games
    try:
        game = chess.pgn.read_game(pgn)
        board = game.board()
        moves = list(game.mainline_moves())
        
        # Player
        players['White_Elo'].append(game.headers['WhiteElo'])
        players['Black_Elo'].append(game.headers['BlackElo'])
        players['White'].append(game.headers['White'])
        players['Black'].append(game.headers['Black'])
        players['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        
        # Games
        games['Game_ID'].append(game.headers['FICSGamesDBGameNo'])
        games['White'].append(game.headers['White'])  # dummy ID
        games['Black'].append(game.headers['Black'])  # dummy ID
        games['White_Elo'].append(game.headers['WhiteElo'])
        games['Black_Elo'].append(game.headers['BlackElo'])
        games['ECO'].append(game.headers['ECO'])
        games['Result'].append(game.headers['Result'])
        games['Date'].append(game.headers['Date'])
        
        # Moves
        fen_pos = []
        acpl = []
        
        moves_log_dict['Game_ID'].append(game.headers['FICSGamesDBGameNo'])  # dummy ID for this dataset only breaks
        moves_log_dict['White'].append(game.headers['White'])  # dummy ID
        moves_log_dict['Black'].append(game.headers['Black'])  # dummy ID
        moves_log_dict['ECO'].append(game.headers['ECO'])
        moves_log_dict['WhiteIsComp'].append(game.headers.get('WhiteIsComp', 'No'))
        moves_log_dict['Result'].append(game.headers['Result'])

        # MOVE CYCLE
        for move in moves:
            board.push(move)
            fen_pos.append(board.fen())
            #stockfish.set_fen_position(board.fen())  # load stockfish with current FEN for eval
#             cpl = stockfish.get_evaluation()['value']/100
#             acpl.append(cpl)
        
        moves_log_dict['FEN_moves'].append(fen_pos)
        #moves_log_dict['cpl'].append(acpl)
        
        game_counter += 1
        if game_counter == 55:  # number of games to read
            break
    except AttributeError:  # no further games to read
        print('No further games to load.')
        break

print(f'{game_counter} games read.')
#TODO takes ~1 sec to process 5 games, too slow.

55 games read.
CPU times: user 562 ms, sys: 6.92 ms, total: 569 ms
Wall time: 568 ms


In [5]:
#generates unique IDs from int
def id_generator(id):
    return uuid.uuid4().int

In [6]:
#generates unique IDs containing digits and characters
def short_id_gen(id):
    return shortuuid.ShortUUID().random(length=15)

## df_players

In [7]:
df_players = pd.DataFrame(players)
df_players.head()

,White,White_Elo,Black,Black_Elo,WhiteIsComp
0,forlat,1970,Geforce,2204,Yes
1,Geforce,2201,forlat,1973,No
2,forlat,1976,Geforce,2198,Yes
3,Geforce,2211,forlat,1963,No
4,forlat,1958,Geforce,2216,Yes


In [8]:
#pgn = open("data/Fics_data_pc_data.pgn", encoding='UTF-8') 

In [9]:
#game = chess.pgn.read_game(pgn)

In [10]:
# def players_id_list(input_df, c_black, c_white): # c_white, c_black - columns of respective df, must be a string
#     black = input_df[c_black]
#     white = input_df[c_white]
#     players_names = list(set(list(black) + list(white)))
#     df_pl = {'Players': players_names}
#     players_id = pd.DataFrame(df_pl)
#     players_id['Player_ID'] = players_id['Players'].apply(id_generator)
#     return players_id

#download list? input_df.to_csv("/notebooks/data/players_id")

In [11]:
players_id = pd.DataFrame({'Players': [], 'Player_ID' : []})

def players_id_list(input_df, players_id):
    #extract black and white columns
    black = list(input_df["Black"]) 
    white = list(input_df["White"])
    
    #merge uniqe values from both columns:
    bw_merged = pd.DataFrame(list(set(black + white)), columns=["Players"])
    
    # Player_ID filled with NaNs:
    players_id = players_id.merge(bw_merged, how="outer", left_on=["Players"], right_on=["Players"])
    
    # NaNs replaced with generated IDs
    nans_to_ids = players_id["Player_ID"].fillna(players_id["Player_ID"].apply(id_generator))
    
    #inserting missing IDs to players_id
    players_id["Player_ID"] = nans_to_ids 
    return players_id

In [12]:
players_id = players_id_list(df_players, players_id)

In [13]:
players_id

,Player_ID,Players
0,99946858219252958202651540604265830230,exeComp
1,95448249578145023828495438780702704183,RamMiguel
2,54554719698276717842423088067531859735,Qiyas
3,139908335022802822205456270246572547264,GriffySr
4,69236585082361072931198504058079927466,nakshatra
5,37275849778668080718666026356122144921,Ghannoum
6,721781711190694176538126155661736074,scalaQueen
7,261485049371617504483823207440677253883,iaquinuchess
8,16174104516971638618388076080322425969,Ruvarashe
9,325023012441375470418552373935362463345,chesspickle


In [14]:
df_dummy = pd.DataFrame({'White' : ["12345", 'DummyName', "1234", "forlat", "Geforce"], "Black" : ['DummyName', "12345", "Dummy", "Geforce", "Bambi"]})

In [15]:
df_dummy

,White,Black
0,12345,DummyName
1,DummyName,12345
2,1234,Dummy
3,forlat,Geforce
4,Geforce,Bambi


In [16]:
players_id  = players_id_list(df_dummy, players_id)

In [17]:
def assign_player_id(input_df): # returns a df with 2 new columns and assigned player ID
    #print("List od IDs has been generated:")
    #print(players_unique)
    m_white = input_df.merge(players_id, left_on=["White"], right_on=['Players'])   #
    m_white['White_ID'] = m_white['Player_ID']
    m_white.drop(columns=['Players', "Player_ID"], inplace=True)
    m_bw = m_white.merge(players_id, left_on=["Black"], right_on=['Players'])
    m_bw['Black_ID'] = m_bw['Player_ID']
    m_bw.drop(columns=['Players', "Player_ID"], inplace=True)
    df_players = m_bw
    return df_players

In [18]:
assign_player_id(df_dummy)

,White,Black,White_ID,Black_ID
0,12345,DummyName,135702011331552254653449880024973251232,175150302799481370310330606112823458792
1,DummyName,12345,175150302799481370310330606112823458792,135702011331552254653449880024973251232
2,1234,Dummy,86991862693183327389991262283800699584,141445976980712547581553641416186704502
3,forlat,Geforce,87084752409092957270152321458800292008,149761423060116457221025786069496182744
4,Geforce,Bambi,149761423060116457221025786069496182744,176539040669785779243427376995359543168


In [19]:
# Fix it:
def update_and_assign_player_IDs(input_df, players_id):
    df_players = assign_player_id(input_df)
    return df_players

In [20]:
update_and_assign_player_IDs(df_dummy, players_id)

,White,Black,White_ID,Black_ID
0,12345,DummyName,135702011331552254653449880024973251232,175150302799481370310330606112823458792
1,DummyName,12345,175150302799481370310330606112823458792,135702011331552254653449880024973251232
2,1234,Dummy,86991862693183327389991262283800699584,141445976980712547581553641416186704502
3,forlat,Geforce,87084752409092957270152321458800292008,149761423060116457221025786069496182744
4,Geforce,Bambi,149761423060116457221025786069496182744,176539040669785779243427376995359543168


## df_games

In [57]:
df_games = pd.DataFrame(games)
df_games['Game_ID'] = df_games['Game_ID'].apply(short_id_gen)
##TODO date to datetime
df_games

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result
0,97gUFksxvXsEBkW,2021.01.31,forlat,1970,Geforce,2204,A01,0-1
1,48WGHqCKoqcCGuH,2021.01.31,Geforce,2201,forlat,1973,B00,1-0
2,789568pXQTj5ch5,2021.01.31,forlat,1976,Geforce,2198,A20,0-1
3,7aS8GZQS2Yf28t3,2021.01.31,Geforce,2211,forlat,1963,B10,0-1
4,6wBvKBwULsPgERV,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2
5,BW2QD69yFwm2cHH,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0
6,3Li3hGeDuLHBvMJ,2021.01.31,Geforce,2213,forlat,1961,B00,1-0
7,BGgN9BUvkhEfQW7,2021.01.31,forlat,1964,Geforce,2210,C28,0-1
8,3yLtUm4noYj3otz,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1
9,3CBGtF2LPfdX2aX,2021.01.31,Geforce,2207,forlat,1967,B10,1-0


In [63]:
dummy_games = df_games

In [70]:
dummy_games.iloc[0] = ['jePsyVtxwueeBoR', '2021.01.31', 'Geforce', '2207', 'forlat', '2204', 'C44', '0-1']
dummy_games.iloc[1] = ['jePsyVtxwueeBoR', '2021.01.31', 'Bambi', '1109', 'forlat', '2204', 'C44', '0-1']

In [71]:
dummy_games

,Game_ID,Date,White,White_Elo,Black,Black_Elo,ECO,Result
0,jePsyVtxwueeBoR,2021.01.31,Geforce,2207,forlat,2204,C44,0-1
1,jePsyVtxwueeBoR,2021.01.31,Bambi,1109,forlat,2204,C44,0-1
2,789568pXQTj5ch5,2021.01.31,forlat,1976,Geforce,2198,A20,0-1
3,7aS8GZQS2Yf28t3,2021.01.31,Geforce,2211,forlat,1963,B10,0-1
4,6wBvKBwULsPgERV,2021.01.31,forlat,1958,Geforce,2216,D30,1/2-1/2
5,BW2QD69yFwm2cHH,2021.01.31,exeComp,2717,Ruvarashe,1976,B92,1-0
6,3Li3hGeDuLHBvMJ,2021.01.31,Geforce,2213,forlat,1961,B00,1-0
7,BGgN9BUvkhEfQW7,2021.01.31,forlat,1964,Geforce,2210,C28,0-1
8,3yLtUm4noYj3otz,2021.01.31,Ruvarashe,1976,exeComp,2717,C11,0-1
9,3CBGtF2LPfdX2aX,2021.01.31,Geforce,2207,forlat,1967,B10,1-0


In [72]:
def find_and_replace_duplicates(df, subset):
    while True:
        if df.duplicated(subset=subset, keep=False).any() == True:
            is_dup = df.duplicated(subset=subset, keep="first")
            if subset == 'Game_ID':
                df[subset] = df[subset].where(~is_dup, df[subset].apply(short_id_gen))
            return df
            df[subset] = df[subset].where(~is_dup, df[subset].apply(id_generator))
        else: print(f'\u2713 no duplicates in {subset}s')
        break

In [74]:
find_and_replace_duplicates(dummy_games, 'Game_ID')

✓ no duplicates in Game_IDs


In [ ]:
timestamp = game.headers["Time"]

In [ ]:
df_games

## df_moves

In [ ]:
df_moves = pd.DataFrame(moves_log_dict)
#TODO moves and cpl as features when loading to DB
df_moves['Game_ID'] = df_games['Game_ID']
df_moves.head(20)

In [ ]:
df_moves['Move_ID'] = df_moves.apply(lambda row: f"{row.Game_ID }-{row.White}-{row.Black}", axis=1)

In [ ]:
df_moves['FEN_moves'][7]

# search

In [ ]:
def search_df(df, column, value):
    try:
        index = np.where(df[column] == value)
        df = df.iloc[index]
        if len(df) == 0:
            return f'No games found for {value}.'
        return df
    except KeyError as e:
        print(e, f'not found.')